In [ ]:
# Import EfficientNet models with intermediate endpoints
import sys
sys.path.append('/kaggle/input/efficientnetv2-head-1x1-endpoint-v2/')
sys.path.append('/kaggle/input/efficientnetv2-head-1x1-endpoint-v2/efficientnetv2/')

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow.keras.backend as K
import tensorflow_addons as tfa
import matplotlib.pyplot as plt

from kaggle_datasets import KaggleDatasets
from tqdm.notebook import tqdm
from multiprocessing import cpu_count
from sklearn import metrics
from sklearn.model_selection import StratifiedKFold
from albumentations import *

import effnetv2_model
import re
import os
import io
import time
import pickle
import math
import random
import sys
import cv2
import gc

import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 150

print(f'tensorflow version: {tf.__version__}')
print(f'tensorflow keras version: {tf.keras.__version__}')
print(f'python version: P{sys.version}')

In [ ]:
# Detect hardware, return appropriate distribution strategy
try:
    TPU = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection. No parameters necessary if TPU_NAME environment variable is set. On Kaggle this is always the case.
    print('Running on TPU ', TPU.master())
except ValueError:
    print('Running on GPU')
    TPU = None

if TPU:
    IS_TPU = True
    tf.config.experimental_connect_to_cluster(TPU)
    tf.tpu.experimental.initialize_tpu_system(TPU)
    strategy = tf.distribute.experimental.TPUStrategy(TPU)
else:
    IS_TPU = False
    strategy = tf.distribute.get_strategy() # default distribution strategy in Tensorflow. Works on CPU and single GPU.

REPLICAS = strategy.num_replicas_in_sync
print(f'REPLICAS: {REPLICAS}, IS_TPU: {IS_TPU}')

In [ ]:
# For TPU's the dataset needs to be stored in Google Cloud
# Retrieve the Google Cloud location of the dataset
GCS_DS_PATH = KaggleDatasets().get_gcs_path('hubmap-patched-tfrecords-300x300')

In [ ]:
SEED = 43
DEBUG = False

# Image dimensions
IMG_SIZE_FULL = 640
IMG_SIZE = 640
N_PATCHES_PER_IMAGE = (IMG_SIZE_FULL // IMG_SIZE) ** 2
N_CHANNELS = 3
INPUT_SHAPE = (IMG_SIZE, IMG_SIZE, N_CHANNELS)

# EfficientNet version, b0/b1/b2/b3/b4/b5/b6/b7/b8
EFN_SIZE = 'b8'
# Peak Learning Rate
LR_MAX_WHOLE = 8e-4
N_FOLDS = 4
TRAIN_FOLDS = [0]

# Epochs are run 10 at a time due to low training samples
COMBINE_EPOCHS = 4
EPOCHS_WHOLE = 200 // COMBINE_EPOCHS

# Batch size
BATCH_SIZE = 8 * REPLICAS

# Dataset Mean and Standard Deviation
MEAN = np.load('/kaggle/input/hubmap-patched-tfrecords-300x300/MEAN.npy')
STD = np.load('/kaggle/input/hubmap-patched-tfrecords-300x300/STD.npy')

# Tensorflow AUTO flag
AUTO = tf.data.experimental.AUTOTUNE
if TPU:
    NUM_PARALLEL_CALLS = AUTO
else:
    NUM_PARALLEL_CALLS = cpu_count()

print(f'BATCH_SIZE: {BATCH_SIZE}, NUM_PARALLEL_CALLS: {NUM_PARALLEL_CALLS}')

In [ ]:
print(f'MEAN: {MEAN}, STD: {STD}')

# Seed

In [ ]:
# Seed all random number generators
def seed_everything(seed=SEED):
    os.environ['PYTHONHASHSEED'] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)
    

seed_everything()

# Train

In [ ]:
# Train DataFrame
train = pd.read_csv('/kaggle/input/hubmap-organ-segmentation/train.csv')
N_SAMPLES = len(train)
print(f'N_SAMPLES: {N_SAMPLES}')

# Add Ordinal Encoded Organ
train['organ_ordinal'] = train['organ'].astype('category').cat.codes
N_ORGANS = train['organ'].nunique()
ORGANS = sorted(train['organ'].unique())
org_ord2org = dict(enumerate(train['organ'].astype('category').cat.categories))
print(f'N_ORGANS: {N_ORGANS}, ORGANS: {ORGANS}')

display(train.head())
display(train.info())

# Utility Functions

In [ ]:
# One in K chance function
def tf_rand_int(minval, maxval, dtype=tf.int64):
    minval = tf.cast(minval, dtype)
    maxval = tf.cast(maxval, dtype)
    return tf.random.uniform(shape=(), minval=minval, maxval=maxval, dtype=dtype)

def one_in(k):
    return 0 == tf_rand_int(0, k)

# Loss

The following code provides a wide variety of semantic segmentations losses and metrics. Surprisingly, dice loss did perform worse than good old binary crossnetropy. The only code used in this notebook is the IoU metric.

In [ ]:
# Source: https://github.com/shruti-jadon/Semantic-Segmentation-Loss-Functions/blob/master/loss_functions.py
import tensorflow as tf
import tensorflow.keras.backend as K
from tensorflow.keras.losses import binary_crossentropy

beta = 0.25
alpha = 0.25
gamma = 2
epsilon = 1e-5
smooth = 1
threshold=0.50

class Semantic_loss_functions(object):
    def __init__(self):
        print ('semantic loss functions initialized')
        
    def iou(self, y_true, y_pred):
        y_true = tf.cast(y_true, tf.float32)
        y_pred = tf.where(y_pred > threshold, x=1.0, y=0.0)
        y_true_f = K.flatten(y_true)
        y_pred_f = K.flatten(y_pred)
        intersection = K.sum(y_true_f * y_pred_f)
        union = K.sum(y_true_f + y_pred_f) - intersection
        return intersection / (union + epsilon)

    def dice_coef(self, y_true, y_pred):
        y_true = tf.cast(y_true, tf.float32)
        y_true_f = K.flatten(y_true)
        y_pred_f = K.flatten(y_pred)
        intersection = K.sum(y_true_f * y_pred_f, axis=1)
        return (2. * intersection + K.epsilon()) / (
                    K.sum(y_true_f) + K.sum(y_pred_f) + K.epsilon())

    def sensitivity(self, y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        return true_positives / (possible_positives + K.epsilon())

    def specificity(self, y_true, y_pred):
        true_negatives = K.sum(
            K.round(K.clip((1 - y_true) * (1 - y_pred), 0, 1)))
        possible_negatives = K.sum(K.round(K.clip(1 - y_true, 0, 1)))
        return true_negatives / (possible_negatives + K.epsilon())

    def convert_to_logits(self, y_pred):
        y_pred = tf.clip_by_value(y_pred, tf.keras.backend.epsilon(),
                                  1 - tf.keras.backend.epsilon())
        return tf.math.log(y_pred / (1 - y_pred))

    def weighted_cross_entropyloss(self, y_true, y_pred):
        y_pred = self.convert_to_logits(y_pred)
        pos_weight = beta / (1 - beta)
        loss = tf.nn.weighted_cross_entropy_with_logits(logits=y_pred,
                                                        targets=y_true,
                                                        pos_weight=pos_weight)
        return tf.reduce_mean(loss)

    def focal_loss_with_logits(self, logits, targets, alpha, gamma, y_pred):
        weight_a = alpha * (1 - y_pred) ** gamma * targets
        weight_b = (1 - alpha) * y_pred ** gamma * (1 - targets)

        return (tf.math.log1p(tf.exp(-tf.abs(logits))) + tf.nn.relu(
            -logits)) * (weight_a + weight_b) + logits * weight_b

    def focal_loss(self, y_true, y_pred):
        y_pred = tf.clip_by_value(y_pred, tf.keras.backend.epsilon(),
                                  1 - tf.keras.backend.epsilon())
        logits = tf.math.log(y_pred / (1 - y_pred))

        loss = self.focal_loss_with_logits(logits=logits, targets=y_true,
                                      alpha=alpha, gamma=gamma, y_pred=y_pred)

        return tf.reduce_mean(loss)

    def depth_softmax(self, matrix):
        sigmoid = lambda x: 1 / (1 + K.exp(-x))
        sigmoided_matrix = sigmoid(matrix)
        softmax_matrix = sigmoided_matrix / K.sum(sigmoided_matrix, axis=0)
        return softmax_matrix

    def generalized_dice_coefficient(self, y_true, y_pred):
        smooth = 1e0
        y_true = tf.cast(y_true, tf.float32)
        y_true_f = K.flatten(y_true)
        y_pred_f = K.flatten(y_pred)
        intersection = K.sum(y_true_f * y_pred_f)
        return (2. * intersection + smooth) / (
                    K.sum(y_true_f) + K.sum(y_pred_f) + smooth)
        
    def dice_loss(self, y_true, y_pred):
        loss = 1 - self.generalized_dice_coefficient(y_true, y_pred)
        return loss
    
    def dice_loss_symmetric(self, y_true, y_pred):
        loss = 1 - self.generalized_dice_coefficient(y_true, y_pred)
        loss_neg = 1 - self.generalized_dice_coefficient(1 - y_true, 1 - y_pred)
        return 0.50 * (loss + loss_neg)

    def bce_dice_loss(self, y_true, y_pred):
        loss = binary_crossentropy(y_true, y_pred) + \
               self.dice_loss(y_true, y_pred)
        return loss / 2.0

    def confusion(self, y_true, y_pred):
        smooth = 1
        y_pred_pos = K.clip(y_pred, 0, 1)
        y_pred_neg = 1 - y_pred_pos
        y_pos = K.clip(y_true, 0, 1)
        y_neg = 1 - y_pos
        tp = K.sum(y_pos * y_pred_pos)
        fp = K.sum(y_neg * y_pred_pos)
        fn = K.sum(y_pos * y_pred_neg)
        prec = (tp + smooth) / (tp + fp + smooth)
        recall = (tp + smooth) / (tp + fn + smooth)
        return prec, recall

    def true_positive(self, y_true, y_pred):
        smooth = 1
        y_pred_pos = K.round(K.clip(y_pred, 0, 1))
        y_pos = K.round(K.clip(y_true, 0, 1))
        tp = (K.sum(y_pos * y_pred_pos) + smooth) / (K.sum(y_pos) + smooth)
        return tp

    def true_negative(self, y_true, y_pred):
        smooth = 1
        y_pred_pos = K.round(K.clip(y_pred, 0, 1))
        y_pred_neg = 1 - y_pred_pos
        y_pos = K.round(K.clip(y_true, 0, 1))
        y_neg = 1 - y_pos
        tn = (K.sum(y_neg * y_pred_neg) + smooth) / (K.sum(y_neg) + smooth)
        return tn

    def tversky_index(self, y_true, y_pred):
        y_true = tf.cast(y_true, tf.float32)
        y_true_pos = K.flatten(y_true)
        y_pred_pos = K.flatten(y_pred)
        true_pos = K.sum(y_true_pos * y_pred_pos)
        false_neg = K.sum(y_true_pos * (1 - y_pred_pos))
        false_pos = K.sum((1 - y_true_pos) * y_pred_pos)
        alpha = 0.75
        return (true_pos + smooth) / (true_pos + alpha * false_neg + (
                    1 - alpha) * false_pos + smooth)

    def tversky_loss(self, y_true, y_pred):
        return 1 - self.tversky_index(y_true, y_pred)

    def focal_tversky(self, y_true, y_pred, gamme=2.0):
        pt_1 = self.tversky_index(y_true, y_pred)
        return K.pow((1 - pt_1), gamma)

    def log_cosh_dice_loss(self, y_true, y_pred):
        y_true = tf.cast(y_true, tf.float32)
        x = self.dice_loss(y_true, y_pred)
        return tf.math.log((tf.exp(x) + tf.exp(-x)) / 2.0)

In [ ]:
SEMANTIC_LOSS_FUNCTIONS = Semantic_loss_functions()

# Feature Pyramid Network (FPN)

(Pytorch Source)[https://www.kaggle.com/code/thedevastator/training-fastai-baseline#Model}

In [ ]:
def FPN(xs, output_channels, last_layer, debug=False):
    def _conv(x):
        x = tf.keras.layers.ZeroPadding2D(padding=1)(x)
        x = tf.keras.layers.Conv2D(output_channels * 2, 3, padding='SAME', kernel_initializer='he_normal', activation='relu')(x)
        x = tf.keras.layers.BatchNormalization()(x)
        x = tf.keras.layers.ZeroPadding2D(padding=1)(x)
        x = tf.keras.layers.Conv2D(output_channels, 3, padding='SAME', kernel_initializer='he_normal')(x)
        x = tf.image.resize(x, size=target_size, method=tf.image.ResizeMethod.BILINEAR)
        x = tf.nn.relu(x)
        return x

    target_size = last_layer.shape[1:3]
    xs = tf.keras.layers.Concatenate()([_conv(x) for x in xs])
    x = tf.keras.layers.Concatenate()([xs, last_layer])

    if debug:
        return x, xs
    else:
        return x

# Atrous Spatial Pyramid Pooling (ASPP)

(Pytorch Source)[https://www.kaggle.com/code/thedevastator/training-fastai-baseline#Model]

In [ ]:
def ASPP(x, mid_c=320, dilations=[1, 2, 3, 4], out_c=640, debug=False):
    def _aspp_module(x, filters, kernel_size, padding, dilation, groups=1):
        x = tf.keras.layers.ZeroPadding2D(padding=padding)(x)
        x = tf.keras.layers.Conv2D(
                filters=filters,
                kernel_size=kernel_size,
                dilation_rate=dilation,
                groups=1 if IS_TPU else groups,
                kernel_initializer='he_uniform',
            )(x)
        x = tf.keras.layers.BatchNormalization()(x)
        x = tf.nn.relu(x)
        
        return x
    
    x0 = tf.math.reduce_max(x, axis=(1,2), keepdims=True)
    x0 = tf.keras.layers.Conv2D(filters=mid_c, kernel_size=1, strides=1, kernel_initializer='he_uniform', use_bias=False)(x0)
    x0 = tf.keras.layers.BatchNormalization(gamma_initializer=tf.constant_initializer(value=0.25))(x0)
    x0 = tf.nn.relu(x0)
                                  
                                  
    xs = (
        [_aspp_module(x, mid_c, 1, padding=0, dilation=1)] +
        [_aspp_module(x, mid_c, 3, padding=d, dilation=d, groups=4) for d in dilations]
    )
    
    x0= tf.image.resize(x0, size=xs[0].shape[1:3])
    x = tf.keras.layers.Concatenate()([x0] + xs)
    x = tf.keras.layers.Conv2D(filters=out_c, kernel_size=1, kernel_initializer='he_uniform', use_bias=False)(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.nn.relu(x)
                       
    if debug:
        return x, x0, xs
    else:
        return x

# Upsample

In [ ]:
# PixelShuffle implemented in Tensorflow, not used
def PixelShuffle(x, upscale_factor=2):
    _, w, h, c = x.shape
    n = -1

    c_out = c // upscale_factor ** 2
    w_out = w * upscale_factor
    h_out = h * upscale_factor

    x = tf.reshape(x, [-1, upscale_factor, upscale_factor, w, h, c_out])
    x = tf.transpose(x, [0, 3, 1, 4, 2, 5])
    x = tf.reshape(x, [-1, w_out, h_out, c_out])

    return x

In [ ]:
# Inspiration: https://www.tensorflow.org/tutorials/generative/pix2pix#build_an_input_pipeline_with_tfdata
def upsample(x, concat, target_filters, name, conv2dt_kernel_init_max, relu=True, dropout=0, debug=False):
    filters = concat.shape[-1]
    x_up = tf.keras.layers.Conv2DTranspose(
            filters, # Number of Convolutional Filters
            kernel_size=4, # Kernel Size
            strides=2, # Kernel Steps
            padding='SAME', # linear scaling
            name=f'Conv2DTranspose_{name}', # Name of Layer
            kernel_initializer='he_uniform',
            use_bias=False,
        )(x)
    
    concat = tf.keras.layers.BatchNormalization(
        gamma_initializer=tf.constant_initializer(value=0.25),
        name=f'BatchNormalization_{name}'
    )(concat)
    x = tf.keras.layers.Concatenate(name=f'Concatenate_{name}')([x_up, concat])
    x = tf.nn.relu(x)
    
        
    x = tf.keras.layers.Conv2D(target_filters, 3, padding='SAME', kernel_initializer='he_uniform', activation='relu', name=f'Conv2D_1_{name}')(x)
    x = tf.keras.layers.Conv2D(target_filters, 3, padding='SAME', kernel_initializer='he_uniform', name=f'Conv2D_2_{name}')(x)
    
    if relu:
        x = tf.nn.relu(x)
    
    x = tf.keras.layers.Dropout(dropout, name=f'Dropout_{name}')(x)

    if debug:
        return x, x_up, concat
    else:
        return x

# Model

A lot of debug code is present in the model construction, which will be later used to show the values during a forward pass. 

In [ ]:
GCS_WEIGHTS_PATH = KaggleDatasets().get_gcs_path('efficientnetv2-head-1x1-endpoint-v2')

In [ ]:
def get_model(dropout_decoder=0, dropout_cnn=0, file_path=None, lr=1e-3, eps=1e-7, clipnorm=5.0, wd_coef=1e-2, cnn_trainable=True, debug=DEBUG):
    # enable XLA optmizations
    tf.config.optimizer.set_jit(True)
    # Set seed for deterministic weights initialization
    seed_everything()
    
    with strategy.scope():
        # EfficientNetV2 Backbone # 
        cnn = effnetv2_model.get_model(f'efficientnet-{EFN_SIZE}', include_top=False, weights=None if IS_TPU else 'jft', model_config={ 'conv_dropout': dropout_cnn })
        if IS_TPU:
            WEIGHT_PATH = f'{GCS_WEIGHTS_PATH}/noisy_student_efficientnet-{EFN_SIZE}'
            ckpt = tf.train.latest_checkpoint(WEIGHT_PATH)
            cnn.load_weights(ckpt)

        # Inputs, note the names are equal to the dictionary keys in the dataset
        image = tf.keras.layers.Input(INPUT_SHAPE, name='image', dtype=tf.float32)
        image_norm = tf.cast(image, tf.float32) / 255
        image_norm = tf.keras.layers.experimental.preprocessing.Normalization(mean=MEAN, variance=STD, dtype=tf.float32)(image_norm)

        embedding, up6, up5, up4, up3, up2, up1 = cnn(image_norm, with_endpoints=True)
        
                
        dec0 = ASPP(up2)
        
        dec0 = tf.keras.layers.Dropout(0.30)(dec0)


        dec1 = upsample(dec0, up3, up4.shape[-1] * 4, 'upsample1', 0.02, dropout=dropout_decoder)
        dec2 = upsample(dec1, up4, up5.shape[-1] * 2, 'upsample2', 0.02, dropout=dropout_decoder)
        dec3 = upsample(dec2, up5, up6.shape[-1] * 2, 'upsample3', 0.02)
        dec4 = upsample(dec3, up6, 64, 'upsample4', 0.02)
        
        dec_fpn = FPN([dec0, dec1, dec2, dec3], 32, dec4)
        

        # Head
        x = tf.keras.layers.Dropout(0.10)(dec_fpn)
        x = tf.keras.layers.Conv2D(
            filters=1,
            kernel_size=1,
            padding='SAME',
            kernel_initializer=tf.random_normal_initializer(0.00, 0.05),
            activation=None if debug else 'sigmoid',
            name='Conv2D_3_head'
        )(x)
        output = tf.image.resize(x, size=[IMG_SIZE, IMG_SIZE], method=tf.image.ResizeMethod.BILINEAR)

        # We will use the famous Adam optimizer for fast learning
        optimizer = tf.optimizers.Adam(learning_rate=lr, epsilon=eps, clipnorm=clipnorm)

        # Loss
        loss = tf.keras.losses.BinaryCrossentropy()
        
        # Metrics
        metrics = [
            SEMANTIC_LOSS_FUNCTIONS.iou,
            tf.keras.metrics.Precision(),
            tf.keras.metrics.Recall(),
            tf.keras.metrics.AUC(),
            tf.keras.metrics.BinaryAccuracy(),
        ]



        model = tf.keras.models.Model(inputs=image, outputs=[output])
        
        model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

        if file_path:
            print('Loading pretrained weights...')
            model.load_weights(file_path)

        return model

In [ ]:
# Pretrained File Path: '/kaggle/input/sartorius-training-dataset/model.h5'
tf.keras.backend.clear_session()
gc.collect()
    
model = get_model(file_path=None, debug=False)

In [ ]:
# Plot model summary
model.summary()

In [ ]:
tf.keras.utils.plot_model(model, show_shapes=True, show_dtype=True, show_layer_names=True, expand_nested=True)

# Dataset and Augmentations

In [ ]:
# Function to benchmark the dataset
def benchmark_dataset(dataset, num_epochs=3, n_steps_per_epoch=10, bs=BATCH_SIZE):
    start_time = time.perf_counter()
    for epoch_num in range(num_epochs):
        for idx, (images, labels) in enumerate(dataset.take(n_steps_per_epoch + 1)):
            if idx == 0:
                epoch_start = time.perf_counter()
            elif idx == 1 and epoch_num == 0:
                print(f'image shape: {images.shape}, image dtype: {images.dtype}')
            else:
                pass
        epoch_t = time.perf_counter() - epoch_start
        mean_step_t = round(epoch_t / n_steps_per_epoch * 1000, 1)
        n_imgs_per_s = int(1 / (mean_step_t / 1000) * bs)
        print(f'epoch {epoch_num} took: {round(epoch_t, 2)} sec, mean step duration: {mean_step_t}ms, images/s: {n_imgs_per_s}')

In [ ]:
# Plots a batch of images
def show_batch(dataset, rows=4, cols=4):
    imgs, lbls = next(iter(dataset))
    imgs = imgs.numpy()
    # Plot
    fig, axes = plt.subplots(nrows=rows, ncols=cols, figsize=(rows*4, cols*4))
    for r in range(rows):
        for c in range(cols // 2):
            img = imgs[r*cols+c]
            axes[r, c*2].imshow(img)
            axes[r, c*2].set_title(f'std: {img.std():.1f}')
            lbl = lbls[r*cols+c]
            axes[r, c*2+1].imshow(lbl)

In [ ]:
# Decodes the TFRecords
def decode_image(record_bytes, val):
    features = tf.io.parse_single_example(record_bytes, {
        'image': tf.io.FixedLenFeature([], tf.string),
        'mask': tf.io.FixedLenFeature([], tf.string),
        'organ': tf.io.FixedLenFeature([], tf.string),
    })

    image = tf.io.parse_tensor(features['image'], out_type=tf.uint8)
    image = tf.reshape(image, [IMG_SIZE, IMG_SIZE, N_CHANNELS])
       
    mask = tf.io.parse_tensor(features['mask'], out_type=tf.uint8)
    mask = tf.reshape(mask, [IMG_SIZE, IMG_SIZE, 1])
    
    # Ogran
    organ = features['organ']
    
    return image, mask, organ

In [ ]:
def augment_image(image, mask, organ, val, return_organ):
    
    if not val:
        rotations = tf_rand_int(0, 4, dtype=tf.int32)

        image = tf.image.rot90(image, rotations)
        mask = tf.image.rot90(mask, rotations)
        
        if one_in(2):
            image = tf.image.transpose(image)
            mask = tf.image.transpose(mask)
        
        # Pixel Level Augmentations
        if one_in(2):
            image = tf.image.random_hue(image, 0.2)
        if one_in(2):
            image = tf.image.random_saturation(image, 0.80, 1.20)
        if one_in(2):
            image = tf.image.random_contrast(image, 0.80, 1.20)
        if one_in(2):
            image = tf.image.random_brightness(image, 0.10)
        if one_in(2):
            image = tf.image.random_jpeg_quality(image, 75, 100)
        
        # Random Crop
        offset_x = tf.random.uniform([], 0, tf.cast(IMG_SIZE * 0.50, tf.int32), dtype=tf.int32)
        img_size_crop = IMG_SIZE - offset_x
        if offset_x > 0:
            offset_y = tf.random.uniform([], 0, offset_x, dtype=tf.int32)
        else:
            offset_y = tf.constant(0, dtype=tf.int32)

        # Crop
        if one_in(2):
            image = tf.slice(image, [offset_x, offset_y, 0], [img_size_crop, img_size_crop, N_CHANNELS])
            mask = tf.slice(mask, [offset_x, offset_y, 0], [img_size_crop, img_size_crop, 1])
            
            image = tf.image.resize(image, [IMG_SIZE, IMG_SIZE], method=tf.image.ResizeMethod.BICUBIC)
            image = tf.cast(image / tf.reduce_max(image) * 255, tf.uint8)
            
            mask = tf.image.resize(mask, [IMG_SIZE, IMG_SIZE], method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)
        
        # Rotate
        if one_in(2):
            angle = tf.random.uniform([], -45 * math.pi / 180, 45 * math.pi / 180, dtype=tf.float32)
            image = tfa.image.rotate(image, angle, interpolation='bilinear', fill_mode='reflect')
            mask = tfa.image.rotate(mask, angle, interpolation='nearest', fill_mode='reflect')
        
        # Resize
        image = tf.cast(image / tf.reduce_max(image) * 255, tf.uint8)
        
        # Explicit Reshape for TPU
        image = tf.reshape(image, [IMG_SIZE, IMG_SIZE, N_CHANNELS])
        mask = tf.reshape(mask, [IMG_SIZE, IMG_SIZE, 1])

    if return_organ:
        return image, mask, organ
    else:
        return image, mask

In [ ]:
GCS_DS_PATH = KaggleDatasets().get_gcs_path('hubmap-patched-tfrecords-300x300')
print(f'GCS_DS_PATH: {GCS_DS_PATH}')

TFRECORDS_FILE_PATHS = np.array(tf.io.gfile.glob(f'{GCS_DS_PATH}/*.tfrecords'))
TFRECORDS_FILE_PATHS = np.array(
    sorted(TFRECORDS_FILE_PATHS, key=lambda fp: int(fp.split('.')[-2].split('_')[-1]))
)
print(f'Found {len(TFRECORDS_FILE_PATHS)} TFRecords')

In [ ]:
# Check whether file paths are correctly ordered
pd.options.display.max_colwidth = 999
display(pd.DataFrame(TFRECORDS_FILE_PATHS, columns=['File Path']).sample(10, random_state=SEED))

In [ ]:
N_SAMPLES_PER_TFRECORD = np.load('/kaggle/input/hubmap-patched-tfrecords-300x300/N_SAMPLES_PER_TFRECORD.npy')
print(f'N_SAMPLES_PER_TFRECORD shape: {N_SAMPLES_PER_TFRECORD.shape}, dtype: {N_SAMPLES_PER_TFRECORD.dtype}')

In [ ]:
def get_dataset(tfrecord_idxs=None, bs=BATCH_SIZE, idxs=None, return_steps=False, val=False, return_organ=False, debug=False):
    ignore_order = tf.data.Options()
    ignore_order.experimental_deterministic = False
    if IS_TPU:
        npr = 1 if val else AUTO
    else:
        npr = 1 if val else cpu_count()
    
    if tfrecord_idxs is None:
        dataset = tf.data.TFRecordDataset(TFRECORDS_FILE_PATHS, num_parallel_reads=npr)
    else:
        dataset = tf.data.TFRecordDataset(TFRECORDS_FILE_PATHS[tfrecord_idxs], num_parallel_reads=npr)
    
    dataset = dataset.map(
        lambda record_bytes: decode_image(record_bytes, val), num_parallel_calls=AUTO if IS_TPU else 1
    )
    
    # Cache dataset to speedup dataloader
    dataset = dataset.cache()
    
    if not val and not debug:
        dataset = dataset.with_options(ignore_order)
        dataset = dataset.shuffle(128)
        dataset = dataset.repeat()
    
    # Augment
    dataset = dataset.map(
        lambda image, mask, organ: augment_image(image, mask, organ, val, return_organ), num_parallel_calls=npr
    )

    dataset = dataset.batch(bs, drop_remainder=True)
    dataset = dataset.prefetch(AUTO)
    
    if return_steps:
        return dataset, math.ceil(N_SAMPLES_PER_TFRECORD[tfrecord_idxs].sum() / bs)
    else:
        return dataset

In [ ]:
# Benchmark Dataset
benchmark_dataset(get_dataset(debug=True))

In [ ]:
# Sanity check
images, labels = next(iter(get_dataset(debug=True)))
print(f'images shape: {images.shape}, labels shape: {labels.shape}')
print(f'images dtype: {images.dtype}, labels dtype: {labels.dtype}')

percentiles = [0.01, 0.05, 0.10, 0.25, 0.40 ,0.50, 0.60, 0.75, 0.90, 0.95, 0.99]
display(pd.Series(images.numpy().flatten()).describe(percentiles=percentiles).to_frame(name='images').T)
display(pd.Series(labels.numpy().flatten()).value_counts().to_frame(name='labels').T)

In [ ]:
# Show what we will be training on
show_batch(get_dataset(bs=16, debug=True))

In [ ]:
gc.collect()

# Learning Rate Scheduler

An learning rate scheduler is used with a cosine decay and exponential warmup.

In [ ]:
def lrfn(current_step, num_warmup_steps, lr_max, num_cycles=0.50, num_training_steps=EPOCHS_WHOLE):
    
    if current_step < num_warmup_steps:
        return lr_max * 0.50 ** (num_warmup_steps - current_step)
    else:
        progress = float(current_step - num_warmup_steps) / float(max(1, num_training_steps - num_warmup_steps))

        return max(0.0, 0.5 * (1.0 + math.cos(math.pi * float(num_cycles) * 2.0 * progress))) * lr_max

In [ ]:
def plot_lr_schedule(lr_schedule, epochs):
    fig = plt.figure(figsize=(20, 10))
    plt.plot([None] + lr_schedule + [None])
    # X Labels
    x = np.arange(1, epochs + 1)
    x_axis_labels = [i if epochs <= 40 or i % 5 == 0 or i == 1 else None for i in range(1, epochs + 1)]
    plt.xlim([1, epochs])
    plt.xticks(x, x_axis_labels) # set tick step to 1 and let x axis start at 1
    
    # Increase y-limit for better readability
    plt.ylim([0, max(lr_schedule) * 1.1])
    
    # Title
    schedule_info = f'start: {lr_schedule[0]:.1E}, max: {max(lr_schedule):.1E}, final: {lr_schedule[-1]:.1E}'
    plt.title(f'Step Learning Rate Schedule, {schedule_info}', size=18, pad=12)
    
    # Plot Learning Rates
    for x, val in enumerate(lr_schedule):
        if epochs <= 40 or x % 5 == 0 or x is epochs - 1:
            if x < len(lr_schedule) - 1:
                if lr_schedule[x - 1] < val:
                    ha = 'right'
                else:
                    ha = 'left'
            elif x == 0:
                ha = 'right'
            else:
                ha = 'left'
            plt.plot(x + 1, val, 'o', color='black');
            offset_y = (max(lr_schedule) - min(lr_schedule)) * 0.02
            plt.annotate(f'{val:.1E}', xy=(x + 1, val + offset_y), size=12, ha=ha)
    
    plt.xlabel('Epoch', size=16, labelpad=5)
    plt.ylabel('Learning Rate', size=16, labelpad=5)
    plt.grid()
    plt.show()

# Learning rate for encoder
LR_SCHEDULE = [lrfn(step, num_warmup_steps=3, lr_max=LR_MAX_WHOLE, num_cycles=0.50) for step in range(EPOCHS_WHOLE)]
plot_lr_schedule(LR_SCHEDULE, epochs=EPOCHS_WHOLE)

# Training Whole Model

In [ ]:
ORGAN_PER_TFRECORD = np.load('../input/hubmap-patched-tfrecords-300x300/ORGAN_PER_TFRECORD.npy')

In [ ]:
# Learning Rate Callback
lr_callback = tf.keras.callbacks.LearningRateScheduler(lambda step: LR_SCHEDULE[step], verbose=1)

In [ ]:
# Create KFOLDS
FOLDS = StratifiedKFold(n_splits=N_FOLDS, shuffle=True, random_state=SEED)
HISTORIES = dict()
IDXS = { 'train_idxs': [], 'val_idxs': [] }
for fold, (train_idxs, val_idxs) in enumerate(FOLDS.split(X=TFRECORDS_FILE_PATHS, y=ORGAN_PER_TFRECORD)):
    # Only train selected folds
    if fold not in TRAIN_FOLDS:
        continue
    
    IDXS['train_idxs'].append(train_idxs)
    IDXS['val_idxs'].append(val_idxs)
    train_dataset, train_steps_per_epoch = get_dataset(tfrecord_idxs=train_idxs, bs=BATCH_SIZE, return_steps=True)
    val_dataset, val_steps_per_epoch = get_dataset(tfrecord_idxs=val_idxs, bs=N_PATCHES_PER_IMAGE * REPLICAS, return_steps=True, val=True)
    print('=' * 80)
    print(f'FOLD {fold}, train_steps_per_epoch: {train_steps_per_epoch}, val_steps_per_epoch: {val_steps_per_epoch}')
    print('=' * 80)
    
    tf.keras.backend.clear_session()
    gc.collect()
    
    model = get_model(file_path=None, cnn_trainable=True, lr=LR_MAX_WHOLE, eps=1e-5)
    #model = get_model(file_path='../input/hubmap-training-tf-tpu-efficientnet-b8-640640-p/model_0.h5', cnn_trainable=True, lr=LR_MAX_WHOLE, eps=1e-5)
    if fold == TRAIN_FOLDS[0]:
        print(model.summary())
    
    HISTORIES[fold] = model.fit(
        train_dataset,
        steps_per_epoch = train_steps_per_epoch * COMBINE_EPOCHS,
        validation_data = val_dataset,
        epochs = EPOCHS_WHOLE,
        verbose = 2,
        callbacks = [
            lr_callback,
        ],
    )
    
    model.save_weights(f'model_{fold}.h5')
    
    print('\n' * 3)
    
    break

# Training Visualisation

Training and validation predictions are plotted as a sanity check.

In [ ]:
def plot_results(dataset, nrows, ncols=4):
    images, labels = next(iter(dataset))
    
    # Predict Masks
    labels_pred = model(images, training=False)
    
    fig, axes = plt.subplots(nrows=nrows, ncols=ncols, figsize=(ncols*8, nrows*6))
    
    for r, (img, lbl, lbl_pred) in enumerate(zip(images, labels, labels_pred)):
        if r > nrows - 1: # zero indexed
            break
        # Plot Image
        axes[r, 0].imshow(img)
        axes[r, 0].set_title('Image', size=18)
        axes[r, 0].axis(False)
        
        # Mask
        axes[r, 1].imshow(lbl)
        axes[r, 1].set_title('Mask', size=18)
        axes[r, 1].axis(False)
        
        # Predicted Mask with Threshold
        axes[r, 2].imshow(lbl_pred)
        axes[r, 2].set_title('Mask Predicted', size=18)
        axes[r, 2].axis(False)
        
        # Predicted Mask with Threshold
        lbl_pred_th50 =tf.cast(lbl_pred > 0.50, tf.uint8)
        axes[r, 3].imshow(lbl_pred_th50)
        axes[r, 3].set_title('Mask Predicted Threshold 0.50', size=18)
        axes[r, 3].axis(False)

In [ ]:
# Training Visualisation
plot_results(get_dataset(bs=8, idxs=train_idxs), 8)

In [ ]:
# Validation Visualisation
plot_results(get_dataset(bs=8, idxs=val_idxs, val=True), 8)

# Training History

Plot training history metrics to get an understanding on how the model is learning.

In [ ]:
def plot_history_metric(metric, f_best=np.argmax, ylim=None, yscale=None, yticks=None):
    plt.figure(figsize=(20, 10))
    
    for fold, history in HISTORIES.items():
        values = history.history[metric]
        N_EPOCHS = len(values)
        val = 'val' in ''.join(history.history.keys())
        # Epoch Ticks
        if N_EPOCHS <= 20:
            x = np.arange(1, N_EPOCHS + 1)
        else:
            x = [1, 5] + [10 + 5 * idx for idx in range((N_EPOCHS - 10) // 5 + 1)]
        x_ticks = np.arange(1, N_EPOCHS+1)

        # Validation
        if val:
            val_values = history.history[f'val_{metric}']
            val_argmin = f_best(val_values)
            plt.plot(x_ticks, val_values, label=f'val_fold_{fold}')

        # summarize history for accuracy
        plt.plot(x_ticks, values, label=f'train_fold_{fold}')
        argmin = f_best(values)
        plt.scatter(argmin + 1, values[argmin], color='red', s=75, marker='o', label=f'train_best_fold_{fold}')
        if val:
            plt.scatter(val_argmin + 1, val_values[val_argmin], color='purple', s=75, marker='o', label=f'val_best_fold_{fold}')

    plt.title(f'Model {metric}', fontsize=24, pad=10)
    plt.ylabel(metric, fontsize=20, labelpad=10)

    if ylim:
        plt.ylim(ylim)

    if yscale is not None:
        plt.yscale(yscale)
        
    if yticks is not None:
        plt.yticks(yticks, fontsize=16)

    plt.xlabel('epoch', fontsize=20, labelpad=10)        
    plt.tick_params(axis='x', labelsize=8)
    plt.xticks(x, fontsize=16) # set tick step to 1 and let x axis start at 1
    plt.yticks(fontsize=16)
    
    plt.legend(prop={'size': 10})
    plt.grid()
    plt.show()

In [ ]:
plot_history_metric('loss', f_best=np.argmin, yscale='log')

In [ ]:
plot_history_metric('precision', ylim=(0,1), yticks=np.arange(0, 1.1, 0.1))

In [ ]:
plot_history_metric('recall', ylim=(0,1), yticks=np.arange(0, 1.1, 0.1))

In [ ]:
plot_history_metric('iou', ylim=(0,1), yticks=np.arange(0, 1.1, 0.1))

In [ ]:
plot_history_metric('auc', ylim=(0,1), yticks=np.arange(0, 1.1, 0.1))

# Validation Predictions

In [ ]:
# Validation Dataset
dataset = get_dataset(
        tfrecord_idxs=val_idxs,
        bs=len(val_idxs),
        val=True,
        return_organ=True,
        debug=True,
    )

# Validation images, masks and organs
val_images, val_masks, val_organs = next(iter(dataset))
# Cast from Tensorflow to Numpy
val_images = val_images.numpy()
val_masks = val_masks.numpy()
val_organs = val_organs.numpy()
print(f'val_images shape: {val_images.shape}, val_masks shape: {val_masks.shape}, val_organs shape: {val_organs.shape}')

# Validation Predictions
VAL_Y_PREDS = model.predict(val_images, verbose=1, batch_size=len(val_idxs))
print(f'VAL_Y_PREDS shape: {VAL_Y_PREDS.shape}, VAL_Y_PREDS dtype: {VAL_Y_PREDS.dtype}')